In [1]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
import os, sys, pandas as pd

C:\Users\jeanm\Desktop\Ensg\Semestre3\ProjetRecherche\PIR18\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
sys.path.append(os.path.abspath('..'))
from Scripts.toolbox import read_txt_file, filter_similar_entities

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\jeanm\.cache\huggingface\token
Login successful


In [3]:
# Dossier de stockage de toutes les données
results_path = "../Output/DataLM"
os.makedirs(results_path, exist_ok=True)

In [4]:
# Chargement du modèle d'extraction des données
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

In [5]:
def entities_extractions(dossier):
    """
    Extrait les mots-clés de fichiers texte dans un dossier et enregistre les résultats dans des fichiers séparés.

    Paramètres:
        dossier : str
    Chemin vers le dossier contenant les fichiers texte (.txt).
    """

    # Sous dossier de stockage des différents ODD
    odd_number = dossier[-2:]
    dossier_path = f"{results_path}/ODD{odd_number}"
    os.makedirs(dossier_path, exist_ok=True)
    print("------------------------------------------------------------------------------------------------")
    print(f"\nDébut de traitement de l'ODD {odd_number}...")

    # Liste des noms de fichiers texte dans le dossier
    # J'utilise la condition supplémentaire pour éviter de sélectionner les fichiers traités
    txt_files = [fichier for fichier in os.listdir(dossier)
                 if fichier.endswith(".txt") and "keywords" not in fichier]

    keywords_ODD = []
    print("Extraction des mots-clés...\n")
    for txt_file in txt_files:
        # Convertir le fichier texte en une chaîne de caractères
        data = read_txt_file(f"{dossier}/{txt_file}")

        # Configurer le modèle KeyBert pour l'extraction des mots-clés
        kw_model = KeyBERT(model=sentence_model)
        keywords = kw_model.extract_keywords(
            data,
            keyphrase_ngram_range=(1, 2),   # Taille de mots/phrases clé(e)s
            stop_words='english',           # Eliminer les mots usuels en anglais
            use_maxsum=True,                # Eviter de sélectionner trop proches de sens
            nr_candidates=20,               # Nombre d'éléments à étudier
            top_n=10                        # Nombre d'éléments choisis
        )
        
        # Filtrer les entités similaires
        entities_filtered = filter_similar_entities(keywords, threshold=80)

        # Filtrer les résultats par score et cible
        filtered_keywords = [(kw[0], kw[1], txt_file[12:14]) for kw in entities_filtered if kw[1] > 0.5]
        keywords_ODD.extend(filtered_keywords)

        # Enregistrer les mots-clés
        metadata = txt_file.replace(".txt","")
        metadata = metadata.replace("Metadata-", "Indicateur_")
        output_path_file = f"{dossier_path}/{metadata}.txt"
        with open(output_path_file, "w", encoding="utf-8") as f:
            f.write("Mots-clés extraits et scores :\n")
            f.write("----------------------------------------------------\n")
            for keyword in filtered_keywords:
                f.write(f"{keyword[0]} : {keyword[1]:.4f}\n")
            f.write("----------------------------------------------------\n")

    # Stockage des mots-clés dans un dataframe
    df = pd.DataFrame(keywords_ODD, columns=["Mots-cles", "Scores", "Cibles"])

    print("Regroupement des mots-clés par cible...\n")
    # Enregistrer les mots-clés par cibles
    grouped = df.groupby("Cibles")
    for target, group in grouped:
        output_filename = f"{dossier_path}/Cible_{odd_number}-{target}.txt"
        # Écrire les mots-clés et leurs scores dans le fichier texte
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(f"Mots-clés pour la cible {target} :\n")
            f.write("----------------------------------------------------\n")
            for index, row in group.iterrows():
                f.write(f"{row['Mots-cles']} : {row['Scores']:.4f}\n")
            f.write("----------------------------------------------------\n")

    print("Regroupement des mots-clés par ODD...\n")
    df.to_csv(f"{dossier_path}/ODD{odd_number}.csv", index=False)
    
    print(f"...Fin de traitement de l'ODD {odd_number}\n")


# entities_extractions("../MetaData/ODD01")

In [ ]:
if __name__ == "__main__":
    # Exécution de tous les ODD
    path = "../MetaData"
    odd_files = os.listdir(path)
    for odd_file in odd_files:
        entities_extractions(f"{path}/{odd_file}")
        print("----------------------------------------------------------------------------------------------------------")

------------------------------------------------------------------------------------------------

Début de traitement de l'ODD 01...
Extraction des mots-clés...

Indicateur_01-01-01a
Indicateur_01-01-01b
Indicateur_01-02-01
Indicateur_01-02-02
Indicateur_01-03-01a
Indicateur_01-03-01b
Indicateur_01-04-01
Indicateur_01-04-02
Indicateur_01-05-01
Indicateur_01-05-02
Indicateur_01-05-03
Indicateur_01-05-04
Indicateur_01-0a-01
Indicateur_01-0a-02a
Indicateur_01-0a-02b
Indicateur_01-0b-01
Regroupement des mots-clés par cible...

Regroupement des mots-clés par ODD...

...Fin de traitement de l'ODD 01

None
----------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------

Début de traitement de l'ODD 02...
Extraction des mots-clés...

Indicateur_02-01-01
Indicateur_02-01-02
Indicateur_02-02-01
Indicateur_02-02-02a
Indicateur_02-02-02b
Indicateur_02-02-03
In